Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html

To run this Jupyter notebook you need to have CAMB and the CAMB python package installed. In order to install the CAMB python package on your computer follow the instructions in http://camb.readthedocs.io/en/latest/

In [8]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np
#print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
#sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

In [9]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [10]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [11]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

print om0

0.306732450988


In [12]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [13]:
#calculate results for these parameters
results = camb.get_results(pars)

In [14]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=2.0, npoints = 200)

In [15]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [17]:
#Redshift bins
zlist = np.arange(0.1,1.45,0.1)
ztest = zlist[4]
Nzbins = len(zlist)

print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4]
ztest = 0.5
Number of redshift bins = 14


In [18]:
#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)

print rcom(ztest)

1946.72577762


In [19]:
#Define the growth function in LCDM
def get_growth(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

print get_growth(ztest)

0.756249102937


In [20]:
#Get the growth factor 
def Dg_dz(zz):
    return get_growth(zz)/(1+zz)
def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

print Dgz(ztest)

0.770149045252


In [21]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial OmHI from Crighton et al 2015
def OmHI(zc):
    return 0.0004*pow(1+zc,0.6)

print OmHI(ztest)
print bHI(ztest)

0.00051016980025
0.7725


In [22]:
#Construct P(k,μ,z) 
def Pkz(kk,mu,zc):    
    return pow(1+(get_growth(zc)/bHI(zc))*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

print Pkz(0.1,-1.0,ztest)

27350.1542264


In [23]:
#mean brightness temperature [mK]
def Tb(zc):
    return 180.*OmHI(zc)*hubble*pow(1+zc,2)/Ez(zc)

print Tb(ztest)

0.106553098208


In [24]:
#Construct P_HI(k,μ,z) [mK^2]
def PHI(kk,mu,zc):
    return pow(Tb(zc),2)*pow(bHI(zc),2)*Pkz(kk,mu,zc)

In [25]:
#MeerKAT noise specs
Ndishes=64
Ddish=13.5*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

Area=5000.0 #deg^2
fsky=Area/44000
omegatot = Area*pow(pi/180,2)
ttotal = 3650*60*60 #5 months in sec

#Tsys per band [in mK] from the latest specs document
Tsyslist = [23.5*1e3,23.0*1e3,23.0*1e3,28.0*1e3,29.0*1e3,30.0*1e3,28.5*1e3,29.5*1e3,\
            31.0*1e3,33.0*1e3,34.0*1e3,35.0*1e3,37.0*1e3,38.0*1e3] 

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1
dfpix = 50*1e3 #Hz
midfreq = 1420.4e6 #Hz

def dzpix(zc):
    return (1+zc)*dfpix/midfreq
def sigpix(zc,Tsys):
    return Tsys/np.sqrt(dfpix*tobs(zc)) 
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

def Wsq(kk,zc):
    return np.exp(-pow(kk,2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

def Pnoise(kk,zc,Tsys):
    return pow(sigpix(zc,Tsys),2)*Vpix(zc)*pow(Wsq(kk,zc),-1.)

In [26]:
def kmin(zc):
    return 2*pi/np.sqrt(pow(rcom(zc),2)*omegatot)

def kmax(zc):
    return 0.14*pow(1+zc,2/(2+nss)) #non-linear cutoff (Smith et al 2003)

In [28]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

#effective volume going in the Fisher matrix
def Veff(kk,mu,zc):
    return Vsur(zc)*(PHI(kk,mu,zc)/(PHI(kk,mu,zc)+Pnoise(kk,zc,Tsys)))**2

print "%.4g" % Vsur(0.3)

8.782e+08


In [29]:
#Fisher matrix derivatives

def dlnP_dlnOmHIbHI(kk,mu,zc):
    return 2.0

def dlnP_dlnbeta(kk,mu,zc):
    return 2*pow(mu,2)*(get_growth(zc)/bHI(zc))/(1+(get_growth(zc)/bHI(zc))*mu**2)

print dlnP_dlnbeta(0.1,-1.0,ztest)

0.989369807621


In [30]:
Npar = 2
params = ["0:beta","1:OmHIbHI"]

In [31]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [32]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [33]:
mugrid = np.linspace(-1., 1., 200) 

In [34]:
#%%time

#   Fisher matrix   # 

#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    zc = zlist[zi]
    Tsys = Tsyslist[zi]
    kgrid = np.linspace(kmin(zc), kmax(zc), 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    #Covmat = np.zeros(s)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnbeta(kk,mu,zc)
            elif i==1:  return dlnP_dlnOmHIbHI(kk,mu,zc)
            else: print "out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnbeta(kk,mu,zc)
                    elif j==1:  return dlnP_dlnOmHIbHI(kk,mu,zc)
                    else: print "index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
            #print Fishermat[i,j]
    print zc, np.sqrt(linalg.inv(Fishermat)[0,0]), np.sqrt(linalg.inv(Fishermat)[1,1])

0.1 0.0586858039218 0.0142982369343
0.2 0.031594779673 0.00830469089735
0.3 0.029737979795 0.008539108219
0.4 0.040867670035 0.0121384353121
0.5 0.0499343904676 0.0150132698002
0.6 0.0596761791179 0.0180464104399
0.7 0.0668523941712 0.0201944769264
0.8 0.0786654491163 0.0237787075969
0.9 0.0930376341654 0.0280319855246
1.0 0.110123301252 0.0329731676988
1.1 0.126625792638 0.037522043735
1.2 0.144555302261 0.0423350026873
1.3 0.167097687211 0.0484087071275
1.4 0.188875939842 0.0539795851936
